In [46]:
# Install necessary libraries
!pip install pandas nltk

# Importing libraries
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import ipywidgets as widgets
from IPython.display import display

# Ensure NLTK components are downloaded
import nltk
nltk.download('punkt')

print("Setup complete. Ready to load data.")

Setup complete. Ready to load data.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/manuelamiranda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
# Import the Spotipy library
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Initialize Spotipy with Spotify Developer Credentials
client_id = '1075d2afec3d414ca02ec7002f1d5576'  # Replace with your actual Spotify Client ID
client_secret = 'bb1fd201730b484983d952ecbd344b33'  # Replace with your actual Spotify Client Secret

credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=credentials)

In [48]:
# Load data from a CSV file
data = pd.read_csv('/Users/manuelamiranda/Desktop/lyrics_retrieval/music_data.csv')
print("Data loaded successfully!")
print("Sample data:")
data.head()  # Display the first few rows of the dataset to verify it's loaded correctly

Data loaded successfully!
Sample data:


,artist,title,id,release_date,lyrics
0,Ariana Grande,"​thank u, next",4063065,2018-11-03,[Verse 1]\nThought I'd end up with Sean\nBut h...
1,Ariana Grande,7 rings,4067762,2019-01-18,"[Verse 1]\nYeah, breakfast at Tiffany's and bo..."
2,Ariana Grande,​God is a woman,3681280,2018-07-13,"[Chorus]\nYou, you love it how I move you\nYou..."
3,Ariana Grande,Side To Side,2457495,2016-05-20,[Intro: Ariana Grande & Nicki Minaj]\nI've bee...
4,Ariana Grande,​​no tears left to cry,3649172,2018-04-20,"[Intro]\nRight now, I'm in a state of mind\nI ..."


In [49]:
# Function to preprocess data
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize text
    tokens = word_tokenize(text)
    # Stemming using PorterStemmer
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

# Apply preprocessing to the lyrics column
data['processed_lyrics'] = data['lyrics'].apply(preprocess_text)
print("Data preprocessing complete. Here's a preview:")
data[['lyrics', 'processed_lyrics']].head()

Data preprocessing complete. Here's a preview:


,lyrics,processed_lyrics
0,[Verse 1]\nThought I'd end up with Sean\nBut h...,[ vers 1 ] thought i 'd end up with sean but h...
1,"[Verse 1]\nYeah, breakfast at Tiffany's and bo...","[ vers 1 ] yeah , breakfast at tiffani 's and ..."
2,"[Chorus]\nYou, you love it how I move you\nYou...","[ choru ] you , you love it how i move you you..."
3,[Intro: Ariana Grande & Nicki Minaj]\nI've bee...,[ intro : ariana grand & nicki minaj ] i 've b...
4,"[Intro]\nRight now, I'm in a state of mind\nI ...","[ intro ] right now , i 'm in a state of mind ..."


In [50]:
def search_spotify(song_title, artist):
    query = f"track:{song_title} artist:{artist}"
    results = spotify.search(q=query, type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return {
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'preview_url': track['preview_url'],
            'spotify_url': track['external_urls']['spotify']
        }
    else:
        return "No results found on Spotify."

In [51]:
def retrieve_song_and_search_spotify(lyrics_input):
    processed_input = preprocess_text(lyrics_input)
    match = data[data['processed_lyrics'].str.contains(processed_input, na=False)]
    if not match.empty:
        song_title = match.iloc[0]['title']
        artist = match.iloc[0]['artist']
        spotify_info = search_spotify(song_title, artist)
        display_spotify_result(spotify_info)  # Using the new display function
    else:
        display(Markdown("**No matching song found.**"))

In [52]:
from IPython.display import display, HTML, Audio

def display_spotify_result(spotify_info):
    if isinstance(spotify_info, dict):
        html_content = f"<h3>{spotify_info['name']} by {spotify_info['artist']}</h3>"
        html_content += f"<p><strong>Album:</strong> {spotify_info['album']}</p>"
        html_content += f"<a href='{spotify_info['spotify_url']}' target='_blank'>Listen on Spotify</a>"
        if spotify_info['preview_url']:
            display(HTML(html_content))
            display(Audio(spotify_info['preview_url'], autoplay=False))
        else:
            display(HTML(html_content))
    else:
        print(spotify_info)

In [53]:
import ipywidgets as widgets
from IPython.display import display

# Input for lyrics
lyrics_input = widgets.Text(
    value='',
    placeholder='Type lyrics here...',
    description='Lyrics:',
    disabled=False
)

# Button to trigger the search
search_button = widgets.Button(
    description='Search',
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    icon='search'  # FontAwesome icon name (without 'fa-')
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        retrieve_song_and_search_spotify(lyrics_input.value)

search_button.on_click(on_button_clicked)

display(lyrics_input, search_button, output)


Text(value='', description='Lyrics:', placeholder='Type lyrics here...')

Button(button_style='info', description='Search', icon='search', style=ButtonStyle())

Output()

we need more preprocessing, show metrics of the model, try and remove duplicates from the dataset in order to get more nuanced results, look to improve performance in every way possible, refine the data so that it is all fully in English